Подключение к гугл диску и загрузка необходимых библиотек

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd
from itertools import product, repeat
from pymatgen.core import Element, Structure
from tqdm import tqdm
from matminer.utils.data import MagpieData
import matplotlib.pyplot as plt

In [1]:
!pip install scikit-learn==1.4.2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 37.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml-cu12 25.6.0 requires scikit-learn>=1.5, but you have scikit-learn 1.4.2 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.4.2 which is incompatible.


In [2]:
!pip install pymatgen ase dscribe matminer megnet -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━

Определение списка элементарных свойств

In [ ]:
ELEMENTAL_PROPERTIES = ['AtomicWeight', 'MeltingT',
'Electronegativity', 'NpValence',  'NfValence',
'NdUnfilled', 'NUnfilled','GSvolume_pa',
'GSbandgap', 'GSmagmom', 'SpaceGroupNumber', ]

Получение свойств элементов


*   Берёт каждый элемент из списка свойств.
*   Для каждого атомного номера Z (от 1 до 92) достаёт значение свойства.
*   Сохраняет в словарь вида:
      {
        "AtomicWeight": {"H": 1.0, "He": 4.0, ...},
        "Electronegativity": {"H": 2.2, "He": None, ...},
        ...
      }




In [ ]:
def get_magpie_properties():
    MD = MagpieData()
    all_elem_values = []

    for e in ELEMENTAL_PROPERTIES:
        elem_values = []
        for z in range(1, 93):
            elem_values.append(
                MD.get_elemental_property(Element.from_Z(z), e)
            )
        all_elem_values.append(dict(zip([str(Element.from_Z(z)) for z in range(1, 93)], elem_values)))

    return dict(zip(ELEMENTAL_PROPERTIES, all_elem_values))

Функция генерации дескрипторов для структуры


*   Для каждой структуры создаёт вектор признаков.
*   Для каждого свойства берёт среднее значение по всем атомам структуры.
*   Возвращает numpy-массив признаков (размерность = число свойств).



In [ ]:
def get_descriptors(structure):

    feature_vector = []

    for e in ELEMENTAL_PROPERTIES:
        temp_descrs = []

        for i in structure:
          temp_descrs.append(MAGPIE_PROPERTIES[e][str(i.specie)])
        feature_vector.append(sum(temp_descrs)/len(temp_descrs))
    return np.array(feature_vector)


MAGPIE_PROPERTIES = get_magpie_properties()

Загрузка файла с таргетом

In [8]:
df = pd.read_csv("/content/bulk_id.csv")
df.head()

,material_id,K_VRH
0,mp-10003,194.268884
1,mp-10010,175.449907
2,mp-10015,295.077545
3,mp-10021,49.130670
4,mp-10025,256.768081


Обработка структур


*   Для каждого material_id загружается .cif файл (структура кристалла).
*   Сохраняем саму структуру в structures.
*   Считаем её признаки (через get_descriptors) и сохраняем в features.





In [10]:
import os
structures = []
features = []
for id in df.material_id:
  structure = Structure.from_file(os.path.join("/content/drive/MyDrive/bulk_cifs", str(id) + '.cif'))
  structures.append(structure)
  features.append(get_descriptors(structure))

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


Проверка результата - содержимое файла cif и сам дескриптор

In [14]:
structures[0]

Structure Summary
Lattice
    abc : 6.228505 6.228505 5.023228
 angles : 90.0 90.0 90.0
 volume : 194.87248608402456
      A : np.float64(6.228505) np.float64(0.0) np.float64(3.813859355861643e-16)
      B : np.float64(1.0016199964124179e-15) np.float64(6.228505) np.float64(3.813859355861643e-16)
      C : np.float64(0.0) np.float64(0.0) np.float64(5.023228)
    pbc : True True True
PeriodicSite: Nb0 (Nb) (2.076, 0.9495, 2.512) [0.3333, 0.1525, 0.5]
PeriodicSite: Nb1 (Nb) (4.152, 5.279, 2.512) [0.6667, 0.8475, 0.5]
PeriodicSite: Nb2 (Nb) (0.9495, 4.152, 2.512) [0.1525, 0.6667, 0.5]
PeriodicSite: Nb3 (Nb) (5.279, 2.076, 2.512) [0.8475, 0.3333, 0.5]
PeriodicSite: Nb4 (Nb) (2.076, 5.279, 4.504e-16) [0.3333, 0.8475, 0.0]
PeriodicSite: Nb5 (Nb) (5.279, 4.152, 5.775e-16) [0.8475, 0.6667, 0.0]
PeriodicSite: Nb6 (Nb) (0.9495, 2.076, 1.853e-16) [0.1525, 0.3333, 0.0]
PeriodicSite: Nb7 (Nb) (4.152, 0.9495, 3.124e-16) [0.6667, 0.1525, 0.0]
PeriodicSite: Co8 (Co) (0.0, 0.0, 3.767) [0.0, 0.0, 0.75]


In [16]:
get_descriptors(structures[0])

array([7.64407025e+01, 2.40916667e+03, 1.69666667e+00, 3.33333333e-01,
       0.00000000e+00, 4.50000000e+00, 5.83333333e+00, 1.72341667e+01,
       1.28833333e-01, 2.58078533e-01, 2.22833333e+02])